# Datasets and automation

In the notebook we will access a dataset and process it's content using multip`le chained APIs.

## Accessing the Dataset

First we install the library for accessing the dataset.

In [ ]:
!pip install -q datasets


We can load datsets from huggingface using their name. In this example we are going to load the [DreamBank dreams dataset](https://huggingface.co/datasets/DReAMy-lib/DreamBank-dreams-en).

In [ ]:
from datasets import load_dataset

dataset = load_dataset("DReAMy-lib/DreamBank-dreams-en")

Each dataset has it's own structure, so we first print it.

In [ ]:
print(dataset)

In this case there is a subset called 'train' with 5 features for each of the 22415 entries. We are interested in the 'dreams' feature, and to see their content we print the first 5.

In [ ]:
number_of_dreams = 5

for idx in range(number_of_dreams):
  print(dataset['train']['dreams'][idx])

## Using the dataset with the API

We install the library for accessing the replicate API

In [ ]:
!pip install -q replicate

In [ ]:
import replicate

api_token='<paste-your-token-here>'

client = replicate.Client(api_token=api_token)

Now that we have the dataset, we want to convert the dream descriptions into image prompts, and to do that we use the LLM [Llama 3 8b Instruct](https://replicate.com/meta/meta-llama-3-8b-instruct/api).

What we have to do is to create a text prompt that creates the image prompt from the description. In the case of Llama there is also the system prompt that describes the role of the LLM. There is a lot of trial an error in the process.

Apart from the prompts, there are more parameters, you check the [API tab](https://replicate.com/meta/meta-llama-3-8b-instruct/api).

In [ ]:
def create_image_prompt(dream):

  prompt = f"""Below is an instruction that describes a task. Write a short response that appropriately completes the request.

  ### Instruction:
  You are an excellent Stable diffusion prompt writer. Given a one-sentence story, you can write a Stable diffusion prompt.
  Instructions for writing a Stable diffusion prompt with a one-sentence story:
  • Use specific descriptors based on the one-sentence story details (e.g., "dense pine forest" instead of "forest").
  • Detail objects mentioned in the one-sentence story (e.g., "ancient leather-bound book" instead of "book").
  • Highlight emotions conveyed in the one-sentence story (e.g., "melancholic," "joyful").
  • Choose a visual style, such as "35mm film," "cinematic," "watercolor", "oil painting", "3D art".
  • Reference an artist or artistic style that aligns with the one-sentence story vibe.
  • Output only the prompt without any explanation

  Examples of Stable diffusion prompts:
  • "Macie Grey in a Victorian dress, exploring a luminous garden with glowing plants and mystical creatures, rendered in a dreamy watercolor style with soft pastel hues."
  • "Mark Hamel as an astronaut in a futuristic suit, examining an ancient monolith amidst ruins on an alien planet, captured with a 35mm film grain effect and a surreal, otherworldly color palette."
  • "Baby shark with a painted face on an old wall, in the style of hyper-realistic sculptures, fragmented figures, distressed materials, tiled walls of light grays, cracked, rococo—inspired art"
  • "Award-winning cinematic bioluminescent oil creature design in gold, vibrant holographic gradient blue and silver colored scheme, in the style 3d hydro — drip venom character, ray tracing reflection, prismatic lighting, realistic texture detail, vibrant electric flames coursing through oil"
  • "Pope Francis wearing leather jacket is a DJ in a nightclub, mixing live on stage, giant mixing table, 4k resolution, a masterpiece"
  • "Kanye West in medieval armor, standing on a cliff's edge, watching a dragon soar from misty mountains, depicted in a Renaissance painting style with dramatic chiaroscuro lighting."
  • "A trench-coated Dick Tracy, silhouette against neon noir streets, searching for elusive clues of a jewel thief, visualized in a vibrant neon noir style with rain-soaked streets reflecting the city's lights."

  Dream description: {dream}

  ### Response:"""

  input = {
      "system_prompt string": "You are a helpful AI assistant.",
      "prompt": prompt,
      "max_new_tokens": 256,
      "prompt_template": "<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"
  }

  output = client.run(
      "meta/meta-llama-3-8b-instruct",
      input=input
  )

  image_prompt = "".join(output)

  return image_prompt

In [ ]:
number_of_dreams = 5

first_dreams = dataset['train']['dreams'][0:number_of_dreams]

image_prompts = []

for dream in first_dreams:
  print("Dream:", dream)
  image_prompt = create_image_prompt(dream)
  print("Prompt:", image_prompt)
  print("")

  image_prompts.append(image_prompt)

## Generate the images

In [ ]:
#@title Clean the image prompt

import re

def clean_prompt(prompt):

  # https://stackoverflow.com/a/49146722/330558
  def remove_emoji(string):
      emoji_pattern = re.compile("["
                            u"\U0001F600-\U0001F64F"  # emoticons
                            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u"\U0001F680-\U0001F6FF"  # transport & map symbols
                            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                            u"\U00002702-\U000027B0"
                            u"\U000024C2-\U0001F251"
                            "]+", flags=re.UNICODE)
      return emoji_pattern.sub(r'', string)

  clean_image_prompt = remove_emoji(image_prompt)
  # text_between_quotes = re.findall('"([^"]*)"', clean_image_prompt)
  # if len(text_between_quotes) > 0:
  #   clean_image_prompt = text_between_quotes[0]
  print(clean_image_prompt)

  return clean_image_prompt

In [ ]:
clean_image_prompts = []
for image_prompt in image_prompts:
  clean_image_prompts.append(clean_prompt(image_prompt))

In [ ]:
def generate_image(image_prompt):
  input = {
    "prompt": image_prompt,
    "megapixels": "0.25"
  }

  output = client.run(
    "black-forest-labs/flux-schnell",
    input=input
  )

  return output

In [ ]:
from tqdm import tqdm

images = []
for clean_image_prompt in tqdm(clean_image_prompts):
  image = generate_image(clean_image_prompt)
  images.append(image)

Finally, we display the images.

In [ ]:
import requests
from PIL import Image

for image_url in images:
  image = Image.open(requests.get(image_url[0], stream=True).raw)
  display(image)

# Finalizing

When you finish working you have to remember to **stop the runtime**, because there is a time limit and to avoid wasting resources. To stop the runtime click Manage Sessions on the Runtime menu. Once the dialog opens click terminate on the current runtime.

> But when you stop the runtime everything you have not saved is ⚠ **lost** ⚠, so be sure to **download** everything you want to keep before stopping it.


# Credits

Taller Estampa https://tallerestampa.com / https://github.com/estampa
